In [1]:
from astropy.io import fits
from astropy.table import unique, vstack
from astropy.table import Table
from astropy.time import Time
from astroquery.mast import Mast,Observations
import matplotlib.pyplot as plt
import os
import pandas as pd
import glob
import json
import jwst

In [2]:
jwst.__version__

'1.8.3.dev57+g3abbb02'

In [3]:
def set_params(parameters):
    return [{"paramName" : p, "values" : v} for p, v in parameters.items()]

def set_mjd_range(min, max):
    '''Set time range in MJD given limits expressed as ISO-8601 dates'''
    return {
        "min": Time(min, format='isot').mjd, 
        "max": Time(max, format='isot').mjd
        }

In [4]:
def get_proposal_products(proposal_id:str):
    
    service = 'Mast.Jwst.Filtered.Nircam'

    keywords = {
            'exp_type':['NRC_CORON'],
            'instrume':['NIRCAM'],
            'date_obs_mjd': [set_mjd_range('2022-01-01','2022-12-01')],
               }

    params = {
            'columns': '*',
            'filters': set_params(keywords)
             }
    print(service, params)
    t = Mast.service_request(service, params)
    fn = list(set(t['filename']))
    ids = list(set(['_'.join(x.split('_')[:-1]) for x in fn]))
    
    matched_obs = Observations.query_criteria(instrument_name='Nircam',
                                          obs_id=ids,
                                         )
    batch_size = 10
    batches = [matched_obs[i:i+batch_size] for i in range(0, len(matched_obs), batch_size)]
    t = [Observations.get_product_list(obs) for obs in batches]
    products = unique(vstack(t), keys='productFilename')
    filtered_products = Observations.filter_products(products,
                                                 #dataproduct_type='image',
                                                 #productType = ['SCIENCE'],
                                                 proposal_id = [f'{proposal_id}'],
                                                 productSubGroupDescription = [#'UNCAL',
                                                                               'RATEINTS',
                                                                               #'CALINTS',
                                                                               'ASN'])
    
    return products,filtered_products
    

In [5]:
PROPOSAL_ID = '1537'

In [6]:
products,filtered_products = get_proposal_products(proposal_id=PROPOSAL_ID)

Mast.Jwst.Filtered.Nircam {'columns': '*', 'filters': [{'paramName': 'exp_type', 'values': ['NRC_CORON']}, {'paramName': 'instrume', 'values': ['NIRCAM']}, {'paramName': 'date_obs_mjd', 'values': [{'min': 59580.0, 'max': 59914.0}]}]}


In [7]:
len(filtered_products), len(products)

(366, 17482)

In [10]:
#prd_df = filtered_products.to_pandas()

In [8]:
def determine_and_count(column:str,table):    
    prop_list = []
    counter = {}
    for i in range(len(table['proposal_id'])):

        if table[f'{column}'][i] not in prop_list:

            prop_list.append(table[f'{column}'][i])
            try:
                counter[table[f'{column}'][i]] = 1
            except:
                #print(table[f'{column}'][i])
                continue
        else:
            try:
                counter[table[f'{column}'][i]] += 1
            except:
                continue
            
    #print(prop_list)
    print(counter)

In [9]:
determine_and_count('productSubGroupDescription',filtered_products)

{'ASN': 183, 'RATEINTS': 183}


In [10]:
def get_json_table(table,suffix):
    
    testing_df = table.to_pandas()
    product_files = testing_df.loc[testing_df['productSubGroupDescription'] == f'{suffix}']['obs_id']
    obs_ids = []
    for i in product_files:
        obs_ids.append(i)
        
    products_df = testing_df.loc[testing_df['obs_id'].isin(obs_ids)]
    info_files = products_df.loc[products_df['productType']=='INFO']
    info_table = Table.from_pandas(info_files)
    
    return info_table

In [11]:
def get_rateints_files(table):
    
    rateints_df = table.to_pandas()
    rateints_df = rateints_df.loc[rateints_df['productSubGroupDescription'] == 'RATEINTS']
    rateints_table = Table.from_pandas(rateints_df)
    
    return rateints_table

In [12]:
filtered_products

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str9,str4,str5,str51,str64,str1,str83,str9,str28,str14,str1,str7,str5,str4,str65,int64,str9,str16,int64
91537068,JWST,image,jw01537025001_04107_00002_nrca4,source/target (L3) : association generator,S,mast:JWST/product/jw01537-o025_20230113t190744_image2_00001_asn.json,INFO,Minimum Recommended Products,ASN,--,CALJWST,1.6.2,1537,jw01537-o025_20230113t190744_image2_00001_asn.json,1887,91537068,PUBLIC,2
91532274,JWST,image,jw01537025001_04103_00003_nrca4,source/target (L3) : association generator,S,mast:JWST/product/jw01537-o025_20230113t190744_image2_00002_asn.json,INFO,Minimum Recommended Products,ASN,--,CALJWST,1.6.2,1537,jw01537-o025_20230113t190744_image2_00002_asn.json,1887,91532274,PUBLIC,2
91535920,JWST,image,jw01537025001_04109_00002_nrca4,source/target (L3) : association generator,S,mast:JWST/product/jw01537-o025_20230113t190744_image2_00003_asn.json,INFO,Minimum Recommended Products,ASN,--,CALJWST,1.6.2,1537,jw01537-o025_20230113t190744_image2_00003_asn.json,1887,91535920,PUBLIC,2
91535912,JWST,image,jw01537025001_0410b_00002_nrca4,source/target (L3) : association generator,S,mast:JWST/product/jw01537-o025_20230113t190744_image2_00004_asn.json,INFO,Minimum Recommended Products,ASN,--,CALJWST,1.6.2,1537,jw01537-o025_20230113t190744_image2_00004_asn.json,1887,91535912,PUBLIC,2
91537046,JWST,image,jw01537025001_04109_00003_nrca4,source/target (L3) : association generator,S,mast:JWST/product/jw01537-o025_20230113t190744_image2_00005_asn.json,INFO,Minimum Recommended Products,ASN,--,CALJWST,1.6.2,1537,jw01537-o025_20230113t190744_image2_00005_asn.json,1887,91537046,PUBLIC,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91812136,JWST,image,jw01537029001_0310n_00001_nrcalong,exposure (L2a): 3D countrate per integration,S,mast:JWST/product/jw01537029001_0310n_00001_nrcalong_rateints.fits,SCIENCE,Minimum Recommended Products,RATEINTS,--,CALJWST,1.6.2,1537,jw01537029001_0310n_00001_nrcalong_rateints.fits,2119680,91812136,PUBLIC,2
91812070,JWST,image,jw01537029001_0310n_00002_nrcalong,exposure (L2a): 3D countrate per integration,S,mast:JWST/product/jw01537029001_0310n_00002_nrcalong_rateints.fits,SCIENCE,Minimum Recommended Products,RATEINTS,--,CALJWST,1.6.2,1537,jw01537029001_0310n_00002_nrcalong_rateints.fits,2119680,91812070,PUBLIC,2
91812038,JWST,image,jw01537029001_0310n_00003_nrcalong,exposure (L2a): 3D countrate per integration,S,mast:JWST/product/jw01537029001_0310n_00003_nrcalong_rateints.fits,SCIENCE,Minimum Recommended Products,RATEINTS,--,CALJWST,1.6.2,1537,jw01537029001_0310n_00003_nrcalong_rateints.fits,2119680,91812038,PUBLIC,2


# Downloading the Products

In [16]:
#Observations.login(mast_token)

In [ ]:
coron_files = Observations.download_products(filtered_products,download_dir = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/{PROPOSAL_ID}')

In [29]:
def get_coron():
    
    path = '/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/CoronFiles/mastDownload/JWST/'
    coron_exptype = []
    noncoron_exptype = []

    for i in glob.glob(f'{path}/**/*.json'):

        with open(i,'r') as f:
            asn_file = json.load(f)

            fits_name = asn_file['products'][0]['members'][0]['expname']
            if 'nrc_coron' in asn_file['constraints']:
                #print('There is a coron right here')

                coron_exptype.append(fits_name)

            else:
                noncoron_exptype.append(fits_name)
                
    return coron_exptype, noncoron_exptype

In [30]:
coron, noncoron = get_coron()

In [31]:
coron

['jw01075001001_03105_00002_nrca3_rateints.fits',
 'jw01075001001_03105_00005_nrca2_rateints.fits',
 'jw01075001001_03105_00003_nrca1_rateints.fits',
 'jw01075001001_03105_00009_nrca3_rateints.fits',
 'jw01075001001_03105_00003_nrca4_rateints.fits',
 'jw01075001001_03105_00002_nrca4_rateints.fits',
 'jw01075002001_03107_00004_nrca3_rateints.fits',
 'jw01075002001_03107_00004_nrca4_rateints.fits',
 'jw01075002001_03107_00005_nrca1_rateints.fits',
 'jw01075002001_03107_00003_nrca3_rateints.fits',
 'jw01075002001_03107_00005_nrca4_rateints.fits',
 'jw01075002001_03107_00004_nrca2_rateints.fits',
 'jw01075002001_03107_00002_nrca4_rateints.fits',
 'jw01075002001_03107_00002_nrca2_rateints.fits',
 'jw01075002001_03105_00005_nrca4_rateints.fits',
 'jw01075002001_03105_00005_nrca1_rateints.fits',
 'jw01075002001_03105_00004_nrca2_rateints.fits',
 'jw01075002001_03105_00004_nrca1_rateints.fits',
 'jw01075002001_03105_00005_nrca2_rateints.fits',
 'jw01075002001_03105_00003_nrca4_rateints.fits',


In [36]:
def det_asn_type():
    
    path = '/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/CoronFiles/mastDownload/JWST/'
    image2 = []
    image3 = []

    for i in glob.glob(f'{path}/**/*.json'):

        with open(i,'r') as f:
            asn_file = json.load(f)

            fits_name = asn_file['asn_type']
            if 'image3' in asn_file['asn_type']:
                #print('There is a coron right here')
                image3.append(fits_name)
            else:
                image2.append(fits_name)
                
    return image3,image2

In [37]:
image3, image2 = det_asn_type()

In [39]:
len(image2)

906

In [41]:
len(image3)

0

# Some Visualizations

In [230]:
coron_files

Local Path,Status,Message,URL
str130,str8,object,object
CoronFiles/mastDownload/JWST/jw01075001001_03105_00009_nrca4/jw01075-o001_20221014t190114_image2_001_asn.json,COMPLETE,None,None
CoronFiles/mastDownload/JWST/jw01075001001_03105_00009_nrca2/jw01075-o001_20221014t190114_image2_002_asn.json,COMPLETE,None,None
CoronFiles/mastDownload/JWST/jw01075001001_03105_00007_nrca2/jw01075-o001_20221014t190114_image2_003_asn.json,COMPLETE,None,None
CoronFiles/mastDownload/JWST/jw01075001001_03105_00007_nrca4/jw01075-o001_20221014t190114_image2_004_asn.json,COMPLETE,None,None
CoronFiles/mastDownload/JWST/jw01075001001_03105_00008_nrca3/jw01075-o001_20221014t190114_image2_005_asn.json,COMPLETE,None,None
CoronFiles/mastDownload/JWST/jw01075001001_03105_00005_nrca1/jw01075-o001_20221014t190114_image2_006_asn.json,COMPLETE,None,None
...,...,...,...
CoronFiles/mastDownload/JWST/jw01538052001_03103_00002_nrcalong/jw01538-o052_20221017t055317_image2_051_asn.json,COMPLETE,None,None
CoronFiles/mastDownload/JWST/jw01538052001_0310l_00005_nrcalong/jw01538-o052_20221017t055317_image2_052_asn.json,COMPLETE,None,None


In [26]:
coron10751_ = fits.open(coron_files['Local Path'][0])

In [28]:
coron10751_.info()

Filename: CoronFiles/mastDownload/JWST/jw01075001001_03105_00001_nrca1/jw01075001001_03105_00001_nrca1_rateints.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     263   ()      
  1  SCI           1 ImageHDU        76   (2048, 2048, 1)   float32   
  2  ERR           1 ImageHDU        11   (2048, 2048, 1)   float32   
  3  DQ            1 ImageHDU        12   (2048, 2048, 1)   int32 (rescales to uint32)   
  4  INT_TIMES     1 BinTableHDU     24   1R x 7C   [J, D, D, D, D, D, D]   
  5  VAR_POISSON    1 ImageHDU        10   (2048, 2048, 1)   float32   
  6  VAR_RNOISE    1 ImageHDU        10   (2048, 2048, 1)   float32   
  7  ASDF          1 BinTableHDU     11   1R x 1C   [8499B]   


In [31]:
coron10751_sci = coron10751_[1].data

In [39]:
reshaped_im = coron10751_sci.reshape(2048,2048,1)

Yeah another nothing-like image